In [1]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct


In [1]:
# source_file = "https://escoe-website.s3.amazonaws.com/wp-content/uploads/2018/10/05232820/BGSCSV.zip"
# r = requests.get(source_file)
# z = zipfile.ZipFile(io.BytesIO(r.content))
# z.extractall("downloads/")
# os.listdir("downloads/")

In [2]:
from bgs.load_bgs_amounts import load_bgs_amounts

In [3]:
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

           index Sum of Conventionals
0       Mar 1964           19,070.000
1       Apr 1964           19,070.000
2       May 1964           18,691.000
3       Jun 1964           18,691.000
4       Jul 1964           18,691.000
..           ...                  ...
725  30 Aug 2024        1,946,970.116
726  30 Sep 2024        1,938,161.862
727  31 Oct 2024        1,964,737.881
728  29 Nov 2024        1,982,318.500
729  31 Dec 2024        1,996,881.000

[730 rows x 2 columns]


In [4]:
tables.keys()

dict_keys(['Conventionals', 'Sum of Conventionals', 'Index-linked Old-style', 'Index-linked New-style', 'Calculated indexed nominal Old-style', 'Calculated indexed nominal New-style', 'Sum of total conventional and indexed-linked'])

In [5]:
conv = tables['Conventionals']

In [6]:
conv.shape

(730, 428)

In [7]:
conv.head()

,100,200,300,400,500,600,700,800,900,1000,...,32295,32297,32300,32400,32500,32600,32700,32800,32900,33000
Mar 1964,505,274,391,709,978,,749,400,500,533,...,,,374,1909,510,58,2,21,276,482
Apr 1964,505,274,391,709,978,,749,400,500,533,...,,,374,1909,510,58,2,21,276,482
May 1964,Redeemed,Redeemed,391,709,978,,749,400,500,533,...,,,374,1909,510,58,2,21,276,482
Jun 1964,,,391,709,978,,749,400,500,533,...,,,374,1909,510,58,2,21,276,482
Jul 1964,,,391,709,978,,749,400,500,533,...,,,374,1909,510,58,2,21,276,482


In [8]:
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)


In [9]:
conv.head()

,100,200,300,400,500,600,700,800,900,1000,...,32295,32297,32300,32400,32500,32600,32700,32800,32900,33000
Mar 1964,505.0,274.0,391.0,709.0,978.0,0.0,749.0,400.0,500.0,533.0,...,0.0,0.0,374.0,1909.0,510.0,58.0,2.0,21.0,276.0,482.0
Apr 1964,505.0,274.0,391.0,709.0,978.0,0.0,749.0,400.0,500.0,533.0,...,0.0,0.0,374.0,1909.0,510.0,58.0,2.0,21.0,276.0,482.0
May 1964,0.0,0.0,391.0,709.0,978.0,0.0,749.0,400.0,500.0,533.0,...,0.0,0.0,374.0,1909.0,510.0,58.0,2.0,21.0,276.0,482.0
Jun 1964,0.0,0.0,391.0,709.0,978.0,0.0,749.0,400.0,500.0,533.0,...,0.0,0.0,374.0,1909.0,510.0,58.0,2.0,21.0,276.0,482.0
Jul 1964,0.0,0.0,391.0,709.0,978.0,0.0,749.0,400.0,500.0,533.0,...,0.0,0.0,374.0,1909.0,510.0,58.0,2.0,21.0,276.0,482.0


In [10]:
conv.sum(axis=1).head()

Mar 1964    19070.0
Apr 1964    19070.0
May 1964    18691.0
Jun 1964    18691.0
Jul 1964    18691.0
dtype: float64

In [11]:
sum_conv = tables["Sum of Conventionals"]
sum_conv.head()

,index,Sum of Conventionals
0,Mar 1964,"19,070.000"
1,Apr 1964,"19,070.000"
2,May 1964,"18,691.000"
3,Jun 1964,"18,691.000"
4,Jul 1964,"18,691.000"


In [12]:
from bgs.load_bgs_prices import load_prices
price_df=load_prices("downloads/BGSPrices.csv")

In [13]:
price_df.head()

,100,200,300,400,500,600,700,800,900,1000,...,77000,77100,77200,77300,77400,77500,77600,77700,77800,77900
28 Nov 1975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31 Dec 1975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30 Jan 1976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27 Feb 1976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31 Mar 1976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
price_df.index[0]

'28 Nov 1975'

In [15]:
c = pd.Series(pd.to_datetime(price_df.index, format="%d %b %Y")).dt.strftime("%b %Y").values
c[:10]

array(['Nov 1975', 'Dec 1975', 'Jan 1976', 'Feb 1976', 'Mar 1976',
       'Apr 1976', 'May 1976', 'Jun 1976', 'Jul 1976', 'Aug 1976'],
      dtype=object)

In [16]:
c[0]

'Nov 1975'

In [17]:
conv[conv.index==c[0]]

,100,200,300,400,500,600,700,800,900,1000,...,32295,32297,32300,32400,32500,32600,32700,32800,32900,33000
Nov 1975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,365.0,1909.0,292.0,58.0,2.0,21.0,276.0,482.0


In [18]:
price_df[32300]

28 Nov 1975    27.6875
31 Dec 1975     26.375
30 Jan 1976       29.5
27 Feb 1976    28.9375
31 Mar 1976      28.75
                ...   
30 Aug 2024        0.0
30 Sep 2024          0
31 Oct 2024        0.0
29 Nov 2024        0.0
31 Dec 2024          0
Name: 32300, Length: 590, dtype: object

In [19]:
from bgs.load_gilt_details import load_csv_blocks, process_index_linked_stocks

In [20]:
details = load_csv_blocks("downloads/BGSDetails.csv")

In [21]:
details.keys()

dict_keys([])